In [38]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import matplotlib.pyplot as plt 
%matplotlib notebook 
import os
import utils
reload(utils)

<module 'utils' from 'utils.py'>

In [10]:
X,supply,demand=utils.LoadDataset('./training_set/DesignMatrices/')
X=X.drop(['Temperature','PM2.5'],axis=1) #Dropping other two features for now 
Xnp,supplynp,demandnp=X.values,supply.values,demand.values

In [51]:
Xtrain, supplyTrain, demandTrain, Xval, supplyVal, demandVal=utils.SplitByRegion(Xnp,supplynp,demandnp,.7)


In [52]:
print Xtrain.shape, supplyTrain.shape, demandTrain.shape 
print Xval.shape, supplyVal.shape,demandVal.shape 

(99981L, 3L) (99981L,) (99981L,)
(42899L, 3L) (42899L,) (42899L,)


In [53]:
enc=OneHotEncoder(sparse=False)
t=enc.fit_transform(Xtrain[:,0][np.newaxis])

In [54]:
def one_hot_encode(x, n_classes):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
     """
    return np.eye(n_classes)[x]

In [55]:
regID_ohe = one_hot_encode(Xtrain[:,0],n_classes=67)[:,1:]

In [56]:
weatherID_ohe = one_hot_encode(Xtrain[:,2],10)[:,1:]
weatherID_ohe

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [58]:
def myTransform(X, d):
    """One-hot-encodes region number and weather, while raises timeslot to degree d"""
    a = one_hot_encode(X[:,0],n_classes=67)[:,1:]
    c = one_hot_encode(X[:,2],n_classes=10)[:,1:]
    b = X[:,1][:, np.newaxis]
    b = PolynomialFeatures(degree=d).fit_transform(b)
    
    out=np.concatenate((a, b, c), axis=1)
    return out 

In [65]:
bestscore = np.inf 
bestmodel = None
bestdeg = None
for d in np.arange(1,10+1):
    Xd = myTransform(Xtrain, d)
    model=linear_model.RidgeCV(alphas=np.logspace(1,20,5),)
    model.fit(Xd, demandTrain-supplyTrain)
    
    Xd_ = myTransform(Xval, d)
    ypreds = model.predict(Xd_)
    ypreds2 = model.predict(Xd)
    
    score = utils.MeanAbsoluteError(ypreds,demandVal-supplyVal)
    score2 = utils.MeanAbsoluteError(ypreds2,demandTrain-supplyTrain)
    if bestscore > score: 
        bestscore = score 
        bestmodel = model 
        bestdeg = d
    print 'degree = {}, score = {}, score2 = {}'.format(d, score, score2)

degree = 1, score = 10.6658657092, score2 = 10.8194350197
degree = 2, score = 10.973862876, score2 = 11.1300080548
degree = 3, score = 398.852721478, score2 = 398.61904637
degree = 4, score = 7922105.03513, score2 = 7906941.63005
degree = 5, score = 94022567396.5, score2 = 93790076209.0
degree = 6, score = 1.64002121399e+14, score2 = 1.63517807995e+14
degree = 7, score = 4.64241113145e+18, score2 = 4.62517096628e+18
degree = 8, score = 1.17526473999e+23, score2 = 1.16971190753e+23
degree = 9, score = 1.35357585623e+23, score2 = 1.34559823102e+23
degree = 10, score = 2.05813413122e+22, score2 = 2.04369131335e+22


In [68]:
fnames = [x for x in sorted(os.listdir('./test_set/DesignMatrices/')) if x[0]!='.']

for fname in fnames: 
    Xtestpd = pd.read_csv('./test_set/DesignMatrices/'+fname)
    Xtestpd = Xtestpd[Xtestpd.Timeslot <= 51]
    
    Xtest = Xtestpd[['RegionID','Timeslot', 'Weather']].values
    print Xtest
    Xtest = myTransform(Xtest.astype(int), bestdeg)
    
    ypreds = bestmodel.predict(Xtest)
    
    utils.WriteOnKaggleFormat(Xtestpd, ypreds, fname.split('.')[0])
    break 

[[ 51.  45.  nan]
 [ 28.  44.  nan]
 [ 28.  44.  nan]
 ..., 
 [ 46.  44.  nan]
 [ 46.  44.  nan]
 [  8.  43.   4.]]


IndexError: index -2147483648 is out of bounds for axis 0 with size 10

In [63]:
Xtestpd.columns

Index([u'Unnamed: 0', u'Timeslot', u'RegionHash', u'Date', u'RegionID',
       u'Weather', u'Temperature', u'PM2.5'],
      dtype='object')